In [1]:
from utils import preprocess_features, create_graph
from utils import train, _initialise_training, _run_one_epoch

import pandas as pd
import pickle
from dgl.data.utils import save_graphs
import torch

device = 'cuda'

In [2]:
features = pd.read_csv('../data/large_twitch_features.csv')
features = preprocess_features(features)

edges = pd.read_csv('../data/large_twitch_edges.csv')
train_index = edges.index.to_series().sample(frac=0.9)
test_mask = ~edges.index.isin(train_index)
train_edges = edges.loc[train_index]
test_edges = edges.loc[test_mask]
print(train_edges.shape, test_edges.shape)

g, reverse_eids = create_graph(edges=train_edges, nodes=features)

nodes shape, (168114, 30)
(6117801, 2) (679756, 2)
Creating network graph...
number of nodes 168114
number of edges 6117801
making the graph bi-directional...
number of edges 12235602


In [3]:
# previous params: 128, [15, 10], 5
model, dataloader, optimizer = _initialise_training(
        g, reverse_eids, 'cuda',  
        n_hidden=128, learning_rate=2e-6, graph_sampling_size=[25, 15, 10],
        negative_sample_size=10, weight_decay=1e-5, data_batch_size=512
    )

In [4]:
%%time
for epoch in range(15):
    print('epoch', epoch)
    model.train()
    model, optimizer, loss = _run_one_epoch(
        model=model,
        optimizer=optimizer,
        dataloader=dataloader,
        verbose = 2000,
#         n_optimizer_steps=10000
    )
    print('loss', loss)

epoch 0
Loss 2447.21923828125, GPU Mem 1473.340416MB
Loss 1273.1500244140625, GPU Mem 1473.340416MB
Loss 916.4365234375, GPU Mem 1473.340416MB
Loss 757.7429809570312, GPU Mem 1473.340416MB
Loss 558.6323852539062, GPU Mem 1473.340416MB
Loss 581.7922973632812, GPU Mem 1473.340416MB
Loss 476.8440246582031, GPU Mem 1473.340416MB
Loss 456.7908630371094, GPU Mem 1473.340416MB
Loss 432.8096618652344, GPU Mem 1474.479104MB
Loss 426.6023864746094, GPU Mem 1474.479104MB
Loss 382.36688232421875, GPU Mem 1474.479104MB
loss 380.7869567871094
epoch 1
Loss 325.0823974609375, GPU Mem 1474.479104MB
Loss 320.12957763671875, GPU Mem 1474.479104MB
Loss 329.7298889160156, GPU Mem 1474.479104MB
Loss 290.4817199707031, GPU Mem 1474.479104MB
Loss 281.629150390625, GPU Mem 1474.479104MB
Loss 286.8534240722656, GPU Mem 1474.479104MB
Loss 285.7391662597656, GPU Mem 1474.479104MB
Loss 275.6644592285156, GPU Mem 1474.479104MB
Loss 274.3777160644531, GPU Mem 1474.479104MB
Loss 239.43392944335938, GPU Mem 1474.47910

In [5]:
save_graphs('graph.bin', [g])
with open("model.pkl", "wb") as stream:
    pickle.dump(model, stream)
    

with torch.no_grad():
    node_emb = model.inference(g, device='cuda').numpy()
    
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(node_emb, f)

torch.Size([168114, 30])


  0%|          | 0/132 [00:00<?, ?it/s]

torch.Size([168114, 128])


  0%|          | 0/132 [00:00<?, ?it/s]

torch.Size([168114, 128])


100%|██████████| 132/132 [00:21<00:00,  6.28it/s]


## Evaluatoin

### Create evaluation dataset

In [3]:
%%time
def create_evaluation_dataset(test_edges, unique_users):
    # a sequence includes 5 positive samples + 5 negative samples
    from numpy import random
    from tqdm import tqdm
    
    groupby = test_edges.groupby('numeric_id_1')
    unique_users = set(unique_users)

    def _sample_users(sub_df, unique_users):        
        positives = sub_df.loc[:, 'numeric_id_2'].values
        negatives = list(unique_users.difference(positives))
        users = list(random.choice(positives, 5)) + list(random.choice(negatives, 5))

        return users
    from pandarallel import pandarallel
    pandarallel.initialize()
    sampled_users = groupby.parallel_apply(lambda sub_df: _sample_users(sub_df, unique_users))
    return sampled_users


eval_dataset = create_evaluation_dataset(test_edges, edges['numeric_id_2'].unique())
pickle.dump(eval_dataset, open('eval_dataset.pkl', 'wb'))

In [6]:
import pickle
import numpy as np

node_emb = pickle.load(open('embeddings.pkl', 'rb'))
model = pickle.load(open('model.pkl', 'rb'))
eval_dataset = pickle.load(open('eval_dataset.pkl', 'rb'))


first_player_embeddings = np.vstack([
    np.repeat(node_emb[x].reshape(1, -1), 10, axis=0) for x in eval_dataset.index
])
second_player_embeddings = np.vstack([node_emb[x] for x in eval_dataset])
second_player_embeddings.shape == first_player_embeddings.shape

True

In [7]:
%%time
import torch

predictions = model.predict(
    torch.from_numpy(first_player_embeddings).to(device), 
    torch.from_numpy(second_player_embeddings).to(device)
)
predictions = predictions.view(-1, 10).detach().cpu().numpy()

labels = np.hstack([
    np.ones((eval_dataset.shape[0], 5)),
    np.zeros((eval_dataset.shape[0], 5))
])

labels.shape == predictions.shape

CPU times: user 144 ms, sys: 3.03 ms, total: 147 ms
Wall time: 147 ms


True

### Evaluate: NDCG, AUC

In [8]:
from sklearn.metrics import ndcg_score, roc_auc_score

ndcg = ndcg_score(labels, predictions)
auc = roc_auc_score(labels.flatten(), predictions.flatten())
print('GraphSAGE')
print('ndcg', ndcg)
print('auc', auc)

GraphSAGE
ndcg 0.7880885135280676
auc 0.58878819527459


In [9]:
ndcgs = []
aucs = []
for row in range(labels.shape[0]):
    ndcgs.append(ndcg_score(labels[row].reshape(1, -1), predictions[row].reshape(1, -1)))
    aucs.append(roc_auc_score(labels[row], predictions[row]))

print('GraphSAGE')
print('ndcg', np.mean(ndcgs))
print('auc', np.mean(aucs))

GraphSAGE
ndcg 0.8150392433288077
auc 0.6731400847677065


In [ ]:
%%time
from numpy import random
random_predictions = random.rand(*labels.shape)

ndcgs = []
aucs = []
for row in range(labels.shape[0]):
    ndcgs.append(ndcg_score(labels[row].reshape(1, -1), random_predictions[row].reshape(1, -1)))
    aucs.append(roc_auc_score(labels[row], random_predictions[row]))
    
print('ndcg', np.mean(ndcgs))
print('auc', np.mean(aucs))

ndcg 0.770862998527735
auc 0.5006931327255134
CPU times: user 1min 33s, sys: 127 ms, total: 1min 33s
Wall time: 1min 34s


In [46]:
%%time
perfect_predictions = 1 / (np.ones(labels.shape) * range(1, 11)

ndcgs = []
aucs = []
for row in range(labels.shape[0]):
    ndcgs.append(ndcg_score(labels[row].reshape(1, -1), perfect_predictions[row].reshape(1, -1)))
    aucs.append(roc_auc_score(labels[row], perfect_predictions[row]))
    
print('ndcg', np.mean(ndcgs))
print('auc', np.mean(aucs))

ndcg 1.0
auc 1.0
CPU times: user 1min 32s, sys: 23.8 ms, total: 1min 32s
Wall time: 1min 33s


In [ ]:
# Theoretical best performance: 1
# Theoretical worst performance: 0

# Predict Example

In [1]:
import pickle
from utils import load_graph

model = pickle.load(open('model.pkl', 'rb'))
g = load_graph('graph.bin')

In [2]:
import torch

with torch.no_grad():
    node_emb = model.inference(g, device='cuda').numpy()

torch.Size([168114, 30])


  0%|          | 0/132 [00:00<?, ?it/s]

torch.Size([168114, 128])


  1%|          | 1/132 [00:00<00:43,  2.99it/s]

[Block(num_src_nodes=49920, num_dst_nodes=1280, num_edges=103694)]
[Block(num_src_nodes=49699, num_dst_nodes=1280, num_edges=102737)]


...

[Block(num_src_nodes=46981, num_dst_nodes=1280, num_edges=98204)]
[Block(num_src_nodes=47262, num_dst_nodes=1280, num_edges=97448)]


 99%|█████████▉| 131/132 [00:19<00:00,  5.60it/s]

[Block(num_src_nodes=47884, num_dst_nodes=1280, num_edges=97750)]
[Block(num_src_nodes=47860, num_dst_nodes=1280, num_edges=94143)]


100%|██████████| 132/132 [00:19<00:00,  6.90it/s]

[Block(num_src_nodes=19935, num_dst_nodes=434, num_edges=30028)]


In [6]:
g.edges()

(tensor([     0,      0,      0,  ..., 168112, 168113, 168113]),
 tensor([ 10441,  10464,  13048,  ..., 166575,  96676, 112156]))

In [28]:
g.ndata['feat'][0]

tensor([7.8790e+03, 1.0000e+00, 9.6900e+02, 0.0000e+00, 1.0000e+00, 2.0160e+03,
        2.0000e+00, 2.0180e+03, 1.0000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [29]:
g.ndata['feat'][10441]

tensor([5.8640e+03, 0.0000e+00, 1.1340e+03, 0.0000e+00, 0.0000e+00, 2.0150e+03,
        9.0000e+00, 2.0180e+03, 1.0000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [16]:
device='cuda'
model.predict(
    torch.from_numpy(node_emb[0]).view(1, -1).to(device), 
    torch.from_numpy(node_emb[10441]).view(1, -1).to(device)
)

tensor([[5.7700]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [21]:
model.predict(
    torch.from_numpy(node_emb[0]).view(1, -1).to(device), 
    torch.from_numpy(node_emb[1]).view(1, -1).to(device)
)

tensor([[-5207.7925]], device='cuda:0', grad_fn=<AddmmBackward0>)

# Visualisations

In [ ]:
import pandas as pd

edges = pd.read_csv('../data/large_twitch_edges.csv')

In [ ]:
sample_edges = edges.sample(100)

import networkx as nx
G = nx.from_pandas_edgelist(
    sample_edges,
    source='numeric_id_1',
    target='numeric_id_2'
)

nx.draw_networkx(G)

In [ ]:
import networkx as nx
from pyviz.network import Network

G = nx.from_pandas_edgelist(
    sample_edges,
    source='numeric_id_1',
    target='numeric_id_2'
)
net = Network(notebook=True)
net.from_nx(G)
net.show('graph.html')